This notebook follows the **Medallion Architecture** to refine raw transaction data from the **Bronze** layer into a validated, high-integrity **Silver** table. It focuses on header standardization, transaction-level validation, and deduplication to ensure the data is reliable for financial auditing and Gold-layer aggregations.

##Silver Layer: Transactions Transformation
**Notebook Objective:**

 This notebook automates the transition of header-level transaction data into the Silver layer. It enforces data quality gates, redirects malformed records to a quarantine table for auditing, and ensures a "Single Version of Truth" through deduplication.

##1. Initial Data Profiling (Bronze Layer)

Before processing, we use SQL to check the health of the raw data, specifically looking for null values in critical financial and identification columns.

In [0]:
%sql
-- 1. Check for Nulls in critical business columns
SELECT * FROM `vstone-catalog`.bronze_schema.transactions_bronze 
WHERE round(original_amount - discount_applied, 2) != round(final_amount, 2);

-- 2. Validate financial logic: Total amount must be positive
SELECT count(*) FROM `vstone-catalog`.bronze_schema.transactions_bronze 
WHERE transaction_id IS NULL OR store_id IS NULL;

##2. Configuration & Environment Setup

We establish the naming conventions for our Delta tables using Unity Catalog standards (catalog.schema.table) and initialize the destination schema.

In [0]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, DecimalType

# --- 1. CONFIGURATION ---
# Hyphenated catalogs require backticks for Spark SQL compatibility
CATALOG = "`vstone-catalog`"
SILVER_SCHEMA = "silver_schema"
BRONZE_TABLE = f"{CATALOG}.bronze_schema.transactions_bronze"
SILVER_TABLE = f"{CATALOG}.{SILVER_SCHEMA}.silver_transactions"
QUARANTINE_TABLE = f"{CATALOG}.{SILVER_SCHEMA}.quarantine_transactions"

##Bootstrap: Ensure the Silver schema exists before writing
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SILVER_SCHEMA}")

##3. Data Ingestion & Header Normalization
Raw system headers are often inconsistent. We standardize them to snake_case to maintain a uniform schema across the Lakehouse.

In [0]:
# --- 2. LOAD & STANDARDIZE ---
df_bronze = spark.read.table(BRONZE_TABLE)

# Convert all column names to lowercase and replace spaces with underscores
standardized_cols = [col.lower().replace(" ", "_").strip() for col in df_bronze.columns]
df_standardized = df_bronze.toDF(*standardized_cols)

In [0]:
# --- 3. DATA TYPING ---
df_casted = df_standardized.select(
    "transaction_id", "store_id", "payment_method_id", "voucher_id", "user_id",
    F.col("original_amount").cast(DecimalType(18, 2)).alias("original_amount"),
    F.col("discount_applied").cast(DecimalType(18, 2)).alias("discount_applied"),
    F.col("final_amount").cast(DecimalType(18, 2)).alias("final_amount"),
    F.to_timestamp(F.col("created_at")).alias("created_at"),
    "source_file", "load_dt"
)

##4. Quality Gates & Quarantine Pattern
To maintain high data integrity without crashing the pipeline, we use a Quarantine Pattern. Invalid transactions are tagged and diverted for review, while clean records move forward.

In [0]:
# --- 3. QUALITY GATES ---
# Business Rules: 
# 1. Primary keys (transaction_id, store_id) must exist.
# 2. Total amount must be a positive value.
orig = F.coalesce(F.col("original_amount"), F.lit(0))
disc = F.coalesce(F.col("discount_applied"), F.lit(0))
final = F.coalesce(F.col("final_amount"), F.lit(0))

# UPDATED Business Rules: Adding user_id check
math_valid = (orig - disc == final)
id_valid = (F.col("transaction_id").isNotNull()) & (F.col("store_id").isNotNull())
user_valid = (F.col("user_id").isNotNull()) # <--- YOUR NEW RULE

# Combined Mask
valid_mask = math_valid & id_valid & user_valid

# Isolate Malformed Records
# Divert failed records to Quarantine with a reason tag
df_quarantine = df_casted.filter(~valid_mask) \
    .withColumn("calculated_diff", (orig - disc) - final) \
    .withColumn("quarantine_reason", 
        F.when(~id_valid, "MISSING_MANDATORY_ID")
         .when(~user_valid, "NULL_USER_ID") # <--- SPECIFIC REASON
         .otherwise("FINANCIAL_CALCULATION_ERROR")) \
    .withColumn("quarantined_at", F.current_timestamp())

# Filter Clean Records
df_clean = df_casted.filter(valid_mask)

##5. Deduplication & Final Transformation
We use a **Window function** to ensure each transaction_id is unique in the Silver layer, keeping only the most recent version of a transaction based on its load timestamp.

In [0]:
# --- 4. DEDUPLICATION & NORMALIZATION ---
# Logic: Keep the latest record per transaction_id using a descending row rank
window_spec = Window.partitionBy("transaction_id").orderBy(F.col("load_dt").desc())

df_silver_final = df_clean.withColumn("row_rank", F.row_number().over(window_spec)) \
    .filter("row_rank == 1").drop("row_rank")

##6. Atomic Delta Writes & Integrity Constraints
Finally, we write the data to Delta Lake and apply storage-level constraints to act as a permanent "firewall" against bad data.

In [0]:

# --- 5. ATOMIC WRITES ---
# Append quarantined records; Overwrite Silver to maintain the current state of truth

df_quarantine.write.format("delta").mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(QUARANTINE_TABLE)

# Write Silver records
df_silver_final.write.format("delta").mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_TABLE)

# --- 6. APPLY DELTA CONSTRAINTS ---
# Enforce NOT NULL and business logic at the table level
spark.sql(f"ALTER TABLE {SILVER_TABLE} ALTER COLUMN transaction_id SET NOT NULL")
spark.sql(f"ALTER TABLE {SILVER_TABLE} ALTER COLUMN user_id SET NOT NULL") # Added constraint

try:
    spark.sql(f"ALTER TABLE {SILVER_TABLE} ADD CONSTRAINT positive_total CHECK (final_amount >= 0)")
except Exception as e:
    print(f"Constraint positive_total already exists or could not be added: {e}")

# --- 8. DQ SUMMARY REPORT ---
print("-" * 30)
print(f"SILVER LOAD SUMMARY")
print("-" * 30)
print(f"Total Bronze Records:     {df_bronze.count()}")
print(f"Clean Silver Records:    {df_silver_final.count()}")
print(f"Quarantined Records:     {df_quarantine.count()}")
print(f"""Null User IDs Caught: {df_quarantine.filter("quarantine_reason == 'NULL_USER_ID'").count()}""")
print("-" * 30)

In [0]:
%sql
-- Compare revenue between two versions
SELECT 'v1' as version, sum(final_amount) FROM `vstone-catalog`.silver_schema.silver_transactions VERSION AS OF 1
UNION ALL
SELECT 'v2' as version, sum(final_amount) FROM `vstone-catalog`.silver_schema.silver_transactions VERSION AS OF 2;

##Industry Logics & Standards 
**1. Transaction-Level Deduplication**
In high-volume retail, the same transaction might be sent twice due to network retries. Our logic partitionBy("transaction_id").orderBy(desc("load_dt")) ensures that the finance team never double-counts revenue.

**2. Schema Evolution vs. Integrity**
By using .option("overwriteSchema", "true"), we allow for technical flexibility, but the Delta Constraints (SET NOT NULL) ensure that this flexibility never compromises the core data quality required for auditing.

**3. Idempotency**
This notebook is idempotent. Because it overwrites the Silver table with a fresh calculation of the latest truth from Bronze, it can be safely re-run without creating duplicate data.

**4. Data Governance (Quarantine)**
The quarantine table serves as a "Data Quality Dashboard" source. Instead of data simply disappearing, it is stored with metadata (quarantine_reason) so engineers can trace errors back to the source system.